In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline



from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

np.random.seed(seed=42)

In [2]:
import functions.CreatingNewDataframe as CreatingNewDataframe
import functions.GenerateFis as Features
import functions.GenerateMeansWithOffset as MeansOffset


from functions.date_split import split_month_test
from score_submission import our_scorer, scorer


In [3]:
from sklearn import linear_model
from sklearn.base import BaseEstimator, TransformerMixin


In [4]:
df_initial = pd.read_csv('train.csv.zip', parse_dates=['DATE'])
ids = np.unique(df_initial.ATM_ID)
np.random.shuffle(ids)

In [5]:

import functions.GenerateFis as Features

In [6]:
train_df, test_df = split_month_test(df_initial,
                                     test_days=33,
                                     train_days=-1,
                                     unuseddays_in_end=1)

test_df_for_validation = test_df.copy()
y_test = test_df['CLIENT_OUT']
test_df['CLIENT_OUT'] = None

DF_FOR_FEATURES = pd.concat([train_df, test_df] )

df1, df2, df3, df4, df5 = Features.GenerateFisSasha(DF_FOR_FEATURES)

dfs = [df1, df2, df3, df4, df5]
end_of_train = train_df.shape[0]

df_trains = [df.loc[train_df.index] for df in dfs]

test_days = 33

indices1 = test_df.index.values.reshape(-1, 33)[:, :7].reshape(-1)
indices2 = test_df.index.values.reshape(-1, 33)[:, 7:14].reshape(-1)
indices3 = test_df.index.values.reshape(-1, 33)[:, 14:21].reshape(-1)
indices4 = test_df.index.values.reshape(-1, 33)[:, 21:28].reshape(-1)
indices5 = test_df.index.values.reshape(-1, 33)[:, 28:].reshape(-1)

INDICES = [indices1, indices2, indices3, indices4, indices5]

df_tests = [dfs[i].loc[INDICES[i]] for i in range(5)]
df_tests__ = [dfs[i].loc[INDICES[i]][["DATE", "ATM_ID", "CLIENT_OUT"]] for i in range(5)]
y_tests = [y_test.loc[INDICES[i]] for i in range(5)]


/home/progiv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
df_trains[0]

,DATE,ATM_ID,CLIENT_OUT,weekend_dummy,holiday_dummy,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,...,same_day_4_week_ago,mean_4_week_ago,std_4_week_ago,same_day_4_week_ago_plusday,same_day_4_week_ago_minusday,same_day_5_week_ago,mean_5_week_ago,std_5_week_ago,same_day_5_week_ago_plusday,same_day_5_week_ago_minusday
0,2015-01-01,0,91600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-02,0,136500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-03,0,335400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-04,0,379000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-05,0,344100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-06,0,575300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-01-07,0,158200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015-01-08,0,296500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015-01-09,0,453400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015-01-10,0,415900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
for df in df_trains:
    df.dropna(inplace=True)

In [8]:
y_trains = [df_trains[i][['CLIENT_OUT']] for i in range(5)]

for df in df_trains:
    df.drop(['CLIENT_OUT',  'DATE', "ATM_ID"], axis=1, inplace=True)

for df in df_tests:
    df.drop(['CLIENT_OUT',  'DATE', "ATM_ID"], axis=1, inplace=True)

In [9]:
for i in range(5):
    y_trains[i] = y_trains[i].astype(np.float32)
    df_trains[i] = df_trains[i].astype(np.float32)

In [10]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
# list_of_lasso = [
#     RandomForestRegressor(n_jobs=32, max_depth=7, n_estimators=100, criterion="mae"),
#     RandomForestRegressor(n_jobs=32, max_depth=7, n_estimators=100, criterion="mae"),
#     RandomForestRegressor(n_jobs=32, max_depth=7, n_estimators=100, criterion="mae"),
#     RandomForestRegressor(n_jobs=32, max_depth=7, n_estimators=100, criterion="mae"),
#     RandomForestRegressor(n_jobs=32, max_depth=7, n_estimators=100, criterion="mae"),
# ]

In [12]:
from sklearn.linear_model import Lasso

In [13]:
list_of_lasso = [
    Lasso(),
    Lasso(),
    Lasso(),
    Lasso(),
    Lasso(),
]

In [14]:
for i, lasso in  enumerate(list_of_lasso):
    lasso.fit(df_trains[i], y_trains[i])
    df_tests__[i].CLIENT_OUT = lasso.predict(df_tests[i])
    df_tests__[i].CLIENT_OUT[df_tests__[i].CLIENT_OUT < 0] = 0

/home/progiv/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/progiv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [15]:
df_super_final_predict = pd.concat([df_tests__[0],
                                    df_tests__[1],
                                    df_tests__[2],
                                    df_tests__[3],
                                    df_tests__[4],
                                   ])

In [16]:
scorer(df_super_final_predict, test_df_for_validation)

3230.1737146574437

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# rf1 = RandomForestRegressor(n_estimators=300, max_depth=7, n_jobs=32)
# rf2 = RandomForestRegressor(n_estimators=300, max_depth=7, n_jobs=32)
# rf3 = RandomForestRegressor(n_estimators=300, max_depth=7, n_jobs=32)
# rf4 = RandomForestRegressor(n_estimators=300, max_depth=7, n_jobs=32)
# rf5 = RandomForestRegressor(n_estimators=300, max_depth=7, n_jobs=32)

# rf1.fit(df_train_1, y_train_1, )
# rf2.fit(df_train_2, y_train_2, )
# rf3.fit(df_train_3, y_train_3, )
# rf4.fit(df_train_4, y_train_4, )
# rf5.fit(df_train_5, y_train_5, )